In [1]:
import fastai2
from fastai2.basics import *
from fastai2.callback.all import *
from fastai2.vision.all import *

In [2]:
lbl2name = dict(
  n02086240= 'Shih-Tzu',
  n02087394= 'Rhodesian ridgeback',
  n02088364= 'Beagle',
  n02089973= 'English foxhound',
  n02093754= 'Australian terrier',
  n02096294= 'Border terrier',
  n02099601= 'Golden retriever',
  n02105641= 'Old English sheepdog',
  n02111889= 'Samoyed',
  n02115641= 'Dingo'
)

In [3]:
path = untar_data(URLs.IMAGEWOOF); path.ls()

(#2) [/home/lgvaz/.fastai/data/imagewoof/val,/home/lgvaz/.fastai/data/imagewoof/train]

In [4]:
dblock = DataBlock(blocks=(ImageBlock, CategoryBlock),
          get_items=get_image_files,
          splitter=GrandparentSplitter(valid_name='val'),
          get_y=[parent_label, lbl2name.get])

In [5]:
dbunch = dblock.databunch(path,
                          bs=64,
                          item_tfms=[FlipItem(), RandomResizedCrop(128, 0.35)],
                          batch_tfms=[Normalize(*imagenet_stats)])

In [6]:
def _conv_func(ndim=2, transpose=False):
    "Return the proper conv `ndim` function, potentially `transposed`."
    assert 1 <= ndim <=3
    return getattr(nn, f'Conv{"Transpose" if transpose else ""}{ndim}d')

In [7]:
class ConvLayer(nn.Sequential):
    "Create a sequence of convolutional (`ni` to `nf`), ReLU (if `use_activ`) and `norm_type` layers."
    def __init__(self, ni, nf, ks=3, stride=1, padding=None, bias=None, ndim=2, norm_type=NormType.Batch, bn_1st=True,
                 act_cls=defaults.activation, transpose=False, init=nn.init.kaiming_normal_, xtra=None, **kwargs):
#         if padding is None: padding = ((ks-1)//2 if not transpose else 0)
        layers = []
        if padding is None:
            if   ks==1: pass
            elif ks==2: layers.append(nn.ZeroPad2d((1, 0, 1, 0)))
            else:       raise ValueError(f'ks={ks}')
        bn = norm_type in (NormType.Batch, NormType.BatchZero)
        if bias is None: bias = not bn
        conv_func = _conv_func(ndim, transpose=transpose)
        conv = init_default(conv_func(ni, nf, kernel_size=ks, bias=bias, stride=stride, **kwargs), init)
        if   norm_type==NormType.Weight:   conv = weight_norm(conv)
        elif norm_type==NormType.Spectral: conv = spectral_norm(conv)
        layers += [conv]
        act_bn = []
        if act_cls is not None: act_bn.append(act_cls())
        if bn: act_bn.append(BatchNorm(nf, norm_type=norm_type, ndim=ndim))
        if bn_1st: act_bn.reverse()
        layers += act_bn
        if xtra: layers.append(xtra)
        super().__init__(*layers)

In [8]:
class ResBlock(nn.Module):
    "Resnet block from `ni` to `nh` with `stride`"
    @delegates(ConvLayer.__init__)
    def __init__(self, expansion, ni, nh, stride=1, sa=False, sym=False,
                 norm_type=NormType.Batch, act_cls=defaults.activation, ndim=2, **kwargs):
        super().__init__()
        norm2 = NormType.BatchZero if norm_type==NormType.Batch else norm_type
        nf,ni = nh*expansion,ni*expansion
        layers  = [ConvLayer(ni, nh, 2, stride=stride, norm_type=norm_type, act_cls=None, ndim=ndim, **kwargs),
                   ConvLayer(ni, nh, 2, norm_type=norm_type, act_cls=act_cls, ndim=ndim, **kwargs),
                   ConvLayer(ni, nh, 2, norm_type=norm_type, act_cls=act_cls, ndim=ndim, **kwargs),
                   ConvLayer(nh, nf, 2, norm_type=norm2, act_cls=None, ndim=ndim, **kwargs)
        ] if expansion == 1 else [
                   ConvLayer(ni, nh, 1, norm_type=norm_type, act_cls=act_cls, ndim=ndim, **kwargs),
                   ConvLayer(nh, nh, 2, stride=stride, norm_type=norm_type, act_cls=None, ndim=ndim, **kwargs),
                   ConvLayer(nh, nh, 2, norm_type=norm_type, act_cls=act_cls, ndim=ndim, **kwargs),
                   ConvLayer(nh, nf, 1, norm_type=norm2, act_cls=None, ndim=ndim, **kwargs)
        ]
        self.convs = nn.Sequential(*layers)
        self.sa = SimpleSelfAttention(nf,ks=1,sym=sym) if sa else noop
        self.idconv = noop if ni==nf else ConvLayer(ni, nf, 1, act_cls=None, ndim=ndim, **kwargs)
        self.pool = noop if stride==1 else AvgPool(2, ndim=ndim, ceil_mode=True)
        self.act = defaults.activation(inplace=True) if act_cls is defaults.activation else act_cls()

    def forward(self, x): return self.act(self.sa(self.convs(x)) + self.idconv(self.pool(x)))

In [9]:
class XResNet(nn.Sequential):
    def __init__(self, expansion, layers, c_in=3, c_out=1000, sa=False, sym=False, act_cls=defaults.activation):
        stem = []
        sizes = [c_in,16,32,64] if c_in<3 else [c_in,32,32,64]
        for i in range(3):
            stem.append(ConvLayer(sizes[i], sizes[i+1], 2, stride=2 if i==0 else 1, act_cls=None))
            stem.append(ConvLayer(sizes[i+1], sizes[i+1], 2, act_cls=act_cls))

        block_szs = [64//expansion,64,128,256,512] +[256]*(len(layers)-4)
        blocks = [self._make_layer(expansion, block_szs[i], block_szs[i+1], l, 1 if i==0 else 2,
                                  sa = sa if i==len(layers)-4 else False, sym=sym, act_cls=act_cls)
                  for i,l in enumerate(layers)]
        super().__init__(
            *stem,
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
            *blocks,
            nn.AdaptiveAvgPool2d(1), Flatten(),
            nn.Linear(block_szs[-1]*expansion, c_out),
        )
        init_cnn(self)

    def _make_layer(self, expansion, ni, nf, blocks, stride, sa, sym, act_cls):
        return nn.Sequential(
            *[ResBlock(expansion, ni if i==0 else nf, nf, stride if i==0 else 1,
                      sa if i==blocks else False, sym=sym, act_cls=act_cls)
              for i in range(blocks)])

In [10]:
#export
def _xresnet(pretrained, expansion, layers, **kwargs):
    # TODO pretrain all sizes. Currently will fail with non-xrn50
    url = 'https://s3.amazonaws.com/fast-ai-modelzoo/xrn50_940.pth'
    res = XResNet(expansion, layers, **kwargs)
    if pretrained: res.load_state_dict(load_state_dict_from_url(url, map_location='cpu')['model'], strict=False)
    return res

def xresnet18 (pretrained=False, **kwargs): return _xresnet(pretrained, 1, [2, 2,  2, 2], **kwargs)
def xresnet34 (pretrained=False, **kwargs): return _xresnet(pretrained, 1, [3, 4,  6, 3], **kwargs)
def xresnet50 (pretrained=False, **kwargs): return _xresnet(pretrained, 4, [3, 4,  6, 3], **kwargs)
def xresnet101(pretrained=False, **kwargs): return _xresnet(pretrained, 4, [3, 4, 23, 3], **kwargs)
def xresnet152(pretrained=False, **kwargs): return _xresnet(pretrained, 4, [3, 8, 36, 3], **kwargs)

In [11]:
m = xresnet50(c_out=dbunch.c, sa=True, sym=False, act_cls=MishJit); m

XResNet(
  (0): ConvLayer(
    (0): ZeroPad2d(padding=(1, 0, 1, 0), value=0.0)
    (1): Conv2d(3, 32, kernel_size=(2, 2), stride=(2, 2), bias=False)
    (2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (1): ConvLayer(
    (0): ZeroPad2d(padding=(1, 0, 1, 0), value=0.0)
    (1): Conv2d(32, 32, kernel_size=(2, 2), stride=(1, 1), bias=False)
    (2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): MishJit()
  )
  (2): ConvLayer(
    (0): ZeroPad2d(padding=(1, 0, 1, 0), value=0.0)
    (1): Conv2d(32, 32, kernel_size=(2, 2), stride=(1, 1), bias=False)
    (2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (3): ConvLayer(
    (0): ZeroPad2d(padding=(1, 0, 1, 0), value=0.0)
    (1): Conv2d(32, 32, kernel_size=(2, 2), stride=(1, 1), bias=False)
    (2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): MishJit()
  )
  (4): ConvLayer(


In [12]:
def opt_func(*args, **kwargs):
  opt = RAdam(*args, mom=0.95, wd=1e-2, eps=1e-6, **kwargs)
  return Lookahead(opt)

In [13]:
learn = Learner(dbunch, m, LabelSmoothingCrossEntropy(), metrics=[accuracy], opt_func=opt_func)

In [14]:
# learn.summary()

## Results

In [15]:
# 2x2 double no-act 22,306,826 params, 0.66, 0.63, 0.67, 0.63, 0.67 = .65
learn.fit_flat_cos(5, 4e-3, pct_start=.72, wd=1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,1.987579,2.458240,0.306000,01:16
1,1.719931,2.000820,0.354000,01:11
2,1.563770,1.500287,0.544000,01:11
3,1.453181,1.432962,0.608000,01:11
4,1.262453,1.265259,0.676000,01:11


In [15]:
# 2x2 double 22,306,826 params, 0.59, 0.6
learn.fit_flat_cos(5, 4e-3, pct_start=.72, wd=1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,1.992527,2.297310,0.278000,01:17
1,1.791523,2.079929,0.332000,01:11
2,1.664257,1.699669,0.444000,01:11
3,1.546455,1.567675,0.516000,01:11
4,1.362066,1.389904,0.602000,01:09


In [14]:
# 2x2 same, 0.598, 0.618
learn.fit_flat_cos(5, 4e-3, pct_start=.72, wd=1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,1.972858,2.023067,0.314000,01:01
1,1.791142,2.011371,0.326000,00:56
2,1.635349,1.698493,0.450000,00:56
3,1.543532,1.633707,0.526000,00:56
4,1.355390,1.379606,0.618000,00:55


In [9]:
# 3x3 23,547,754 params, 0.65, 0.64, 0.66 = 0.65
learn.fit_flat_cos(5, 4e-3, pct_start=.72, wd=1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,1.954447,2.488014,0.268000,01:02
1,1.783182,1.990843,0.338000,00:55
2,1.613038,1.576200,0.502000,00:52
3,1.464347,1.480333,0.560000,00:58
4,1.259610,1.297685,0.648000,00:59
